# Imports

In [16]:
import pandas as pd
from pathlib import Path
from mikeio1d import Res1D
from mikeio1d.query import QueryDataReach
from plotly import graph_objects as go

# Messstellen

In [17]:
dict_messstellen = {
    'Messstelle 1B': '134111',
    'Messstelle 1C': '123066',
    'Messstelle 2': '141733',
    'Messstelle 3': '122160',
    'Messstelle 4': '130119',
    'Messstelle 5': '131807',
    'Messstelle 6': '128336'
}

# Mike+ Dateien

In [18]:
dict_res1d = {
    '20230725': Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil\05 Berechnungen Grundlagen"
        r"\8 Entwässerungskonzept\Mike+ Modell\04_Modell\IST-Zustand\Richterswil_IST_m1d - Result Files"
        r"\Richterswil_IST_3wae_230725Default_Network_HD.res1d"),
    '20230828': Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil\05 Berechnungen Grundlagen"
        r"\8 Entwässerungskonzept\Mike+ Modell\04_Modell\IST-Zustand\Richterswil_IST_m1d - Result Files"
        r"\Richterswil_IST_4wae_230828Default_Network_HD.res1d")
}

# Plot Validation

In [19]:
date_format = '%d.%m.%Y %H:%M:%S'
dir_messdaten = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil\05 Berechnungen Grundlagen\8 Entwässerungskonzept\Durchflussmessungen Nivus")

for rain_event, path_res1d in dict_res1d.items():
    save_dir = dir_messdaten / 'Plots' / 'Validation' / rain_event
    res1d = Res1D(str(path_res1d))
 
    if not save_dir.exists():
        save_dir.mkdir(parents=True)

    for k, v in dict_messstellen.items():
        file_path = dir_messdaten / f"History_Data_of_{k}.csv"
        data_nivus = pd.read_csv(file_path, sep=';', usecols=[0, 1, 2], parse_dates=[[0,1]], date_format=date_format)
        data_nivus.set_index('Date_Time', inplace=True)

        query = QueryDataReach('Discharge', v, chainage=0)
        data_mike = res1d.read(query)
        data_mike = data_mike * 1e3 # m3/s -> l/s
        date_from = pd.to_datetime(data_mike.index[0])
        date_to = pd.to_datetime(data_mike.index[-1])

        mask = (data_nivus.index >= date_from) & (data_nivus.index <= date_to)
        data_nivus = data_nivus.loc[mask]
        if not data_nivus.empty:
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=data_nivus.index, y=data_nivus.iloc[:,0], name=k))
            fig.add_trace(go.Scatter(x=data_mike.index, y=data_mike.iloc[:,0], name=v))

            fig.update_layout(title=f'Validierung {k} / Mike Link {v}')
            fig.update_xaxes(title='Zeit')
            fig.update_yaxes(title='Durchfluss [l/s]')

            fig.update_layout(font=dict(size=25))

            save_path = save_dir / f"{k}.html"
            fig.write_html(save_path)